<a href="https://colab.research.google.com/github/Pedrohero13/DB_NO_CON_colab/blob/main/citibike.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalación de la librería streamlit

In [2]:
!pip install streamlit


Vamos a cargar una máquina virtual que tenga instalado Ngrok

In [3]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

--2022-02-22 21:44:28--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 18.205.222.128, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  82.0MB/s    in 0.2s    

2022-02-22 21:44:28 (82.0 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]



Ahora cargaremos esta máquina virtual en nuestro archivo de Google Colab

In [15]:
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


Add token ngrok to ngrok suite

In [113]:
get_ipython().system_raw('./ngrok authtoken 24qeTD0luOCmIfQYHBvCGHHddbz_5CF1HTN82FtBvpFrd8yhf')

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


Posteriormente le indicaremos a Ngrok que estará utilizando el puerto 8501 para visualizar nuestra aplicación

In [114]:
get_ipython().system_raw('./ngrok http 8501 &')

Una vez inicializado el puerto, ahora vamos a crear el tunel para que pueda generarse un enlace que nos permita visualizar nuestra aplicación

In [115]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    'import sys, json; print("Execute the next cell and the go to the following URL: " +json.load(sys.stdin)["tunnels"][0]["public_url"])'

Execute the next cell and the go to the following URL: https://a41a-35-245-56-106.ngrok.io


Una vez que ejecutaron la instrucción anterior, verán que aparece un URL que es el que vamos a dar clic para visualizar nuestra aplicación, pero primero tenemos que ejecutar nuestra aplicación

# **Práctica 2 Caso citibike : Dashboard para visualizar datos de citibike**

Ahora creamos nuestra aplicacion, para guardar el archivo nos aseguramos de incluir la sentencia: %%writefile citibike.py

In [116]:
%%writefile citibike.py
import pandas as pd
import numpy as np
import streamlit as st
import codecs
import matplotlib.pyplot as plt

st.title('Citybike')
sidebar = st.sidebar
sidebar.title("MENU")


DATA_URL = ('/content/citibike.csv')
DATE_COLUMN = 'started_at'
@st.cache
def load_data(nrows):
    data = pd.read_csv(DATA_URL, nrows=nrows, encoding_errors='ignore')
    lowercase = lambda x: str(x).lower()
    data.rename(lowercase, axis='columns', inplace=True)
    data[DATE_COLUMN] = pd.to_datetime(data[DATE_COLUMN])
    return data

#Cargar datos si se selecciona en el checkbox
agree = sidebar.checkbox("¿Quieres mostrar todos los datos? ")
if agree:
  data_load_state = st.text('Cargando...')
  data = load_data(2000)
  
  data_load_state.text("Cargado! (using st.cache)")
  st.dataframe(data)

agree = sidebar.checkbox("¿Quieres mostrar los recorridos por hora? ")
if agree:
  data_load_state = st.text('Cargando...')
  data = load_data(2000)
  data_load_state.text("Cargado! (using st.cache)")
  #hacer la grafica
  st.subheader('Numero de recorridos por hora')
  hist_values = np.histogram(data[DATE_COLUMN].dt.hour, bins=24, range=(0,24))[0]
  st.bar_chart(hist_values)



# Some number in the range 0-23
hour_to_filter = sidebar.slider('HORA', 0, 23, 17)
if hour_to_filter:
  data_load_state = st.text('Cargando...')
  data = load_data(2000)
  data_load_state.text("Cargado! (using st.cache)")

  data2_rename = data.rename(columns = {'start_lat': 'lat', 'start_lng': 'lon'}, inplace = False)

  filtered_data = data2_rename[data2_rename[DATE_COLUMN].dt.hour == hour_to_filter]

  st.subheader('Mapa de los recorridos iniciados a las %s:00' % hour_to_filter)
  st.map(filtered_data)




Overwriting citibike.py


Una vez cargado y ejecutado el archivo ahora vamos a proceder a ejecutarlo con el comando streamlit para que se visualice el contenido del mismo.

In [ ]:
!streamlit run /content/citibike.py

2022-02-22 23:23:52.264 INFO    numexpr.utils: NumExpr defaulting to 2 threads.

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.245.56.106:8501

